In [2]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import requests
import xml.etree.ElementTree as ET
from IPython.display import display, HTML
from scipy import interpolate
from yahoo_fin import options as op, stock_info as si

import logging


# Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)

# Create a file handler
handler = logging.FileHandler('error_log.txt')
handler.setLevel(logging.ERROR)

# Create a formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

# Global flag for controlling print statements
enable_print = False

def custom_print(*args, **kwargs):
    global enable_print
    if enable_print:
        print(*args, **kwargs)


In [3]:
from yahoo_fin import options as op, stock_info as si
s = si.get_live_price("^GSPC")
print(s)
l = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
l[:-2]
l[-2]

4783.830078125


c:\Python311\Lib\site-packages\yahoo_fin\stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


5.0

In [4]:
from scipy.stats import norm
from scipy.optimize import brentq
import numpy as np

# Define the parameters
underlyingPrice = 4567.7998046875
strikePrice = 1200
interestRate = 0.01
daysToExpiration = 2
optionPrice = 0.025    
option_type = 'P'

# Define the Black-Scholes function
def black_scholes(sigma):
    d1 = (np.log(underlyingPrice / strikePrice) + (interestRate + 0.5 * sigma**2) * daysToExpiration) / (sigma * np.sqrt(daysToExpiration))
    d2 = d1 - sigma * np.sqrt(daysToExpiration)
    return strikePrice * np.exp(-interestRate * daysToExpiration) * norm.cdf(-d2) - underlyingPrice * norm.cdf(-d1)

# Define the function to find the root of
def root_function(sigma):
    return black_scholes(sigma) - optionPrice

# Use Brent's method to find the implied volatility
implied_volatility, info = brentq(root_function, 0.0001, 1, full_output=True)

if info.converged:
    print(f"Implied Volatility: {implied_volatility}")
else:
    print("The method did not converge to a solution.")

Implied Volatility: 0.2615502045962956


In [31]:
import mibian

# Define the parameters
underlyingPrice = si.get_live_price("^GSPC")
strikePrice = 1200
interestRate = 0.01
daysToExpiration = 2
optionPrice = 0.025 #3360.5 	
option_type = 'None'
volatility = 492.1875 #0.00010 

# Create an instance of the BS class with the parameters
if option_type == 'C':
    print('Call I.V')
    bs = mibian.BS([underlyingPrice, strikePrice, interestRate, daysToExpiration], callPrice=optionPrice)
elif option_type == 'P':
    print('Put I.V')
    bs = mibian.BS([underlyingPrice, strikePrice, interestRate, daysToExpiration], putPrice=optionPrice)
elif volatility != None:
    print('Volatility I.V')
    bs = mibian.BS([underlyingPrice, strikePrice, interestRate, daysToExpiration], volatility=volatility)    
    print(bs.callPrice, bs.putPrice)
else:
    print('Invalid option type')    

# Print the implied volatility
if volatility == None:
    print(bs.impliedVolatility)

Volatility I.V
3569.846137133341 0.015401474274906068


c:\Python311\Lib\site-packages\yahoo_fin\stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


In [5]:
import datetime
import mibian
from yahoo_fin import options as op, stock_info as si
from IPython.display import display, HTML
from dataclasses import dataclass, field
  
@dataclass
class BSParams:
    underlyingPrice: float = field(default=0.0)
    strikePrice: float = field(default=0.0)
    interestRate: float = field(default=0.0)
    daysToExpiration: int = field(default=0)
    price: float = field(default=0.0)
    option_type: str = field(default='C')
    volatility: float = field(default=0.0)


def calculate_call_iv(bs_params, price):
    return mibian.BS(bs_params, callPrice=price).impliedVolatility
    #return mibian.BS(params[:-2], callPrice=params[-2]).impliedVolatility

def calculate_put_iv(bs_params, price):
    return mibian.BS(bs_params, putPrice=price).impliedVolatility
    #return mibian.BS(params[:-2], putPrice=params[-2]).impliedVolatility

def calculate_iv(params: BSParams):
    bs_params = [params.underlyingPrice, params.strikePrice, params.interestRate, params.daysToExpiration]
    # Use the option type to select the function: 0 for call, 1 for put
    return [calculate_call_iv, calculate_put_iv][params.option_type == 'P'](bs_params, params.price)

def calculate_price(params: BSParams):
    bs_params = [params.underlyingPrice, params.strikePrice, params.interestRate, params.daysToExpiration]
    # Use the option type to select the function: 0 for call, 1 for put
    c = mibian.BS(bs_params, volatility=params.volatility)
    return c.callPrice, c.putPrice

# Time Delta
###december_1 = datetime.date(today.year, 12, 1)
# Get today's date
##today = datetime.date.today() - datetime.timedelta(days=0)
# Calculate the number of days to 1st December
# t_december_1 = (december_1 - today).days
# print(t_december_1)
# print(december_1, today)

# Forward day
forward_days = datetime.timedelta(days= 11).days
forward_minutes = forward_days * 24 * 60
print(forward_days, forward_minutes)
#Spot
spot = si.get_live_price("^GSPC")
#Rate 
r, rate_ = np.array(cubic_spline_risk_free_rate([forward_minutes, forward_minutes]))/100
print(r, rate_)
#r = 0.01
 
#Get data frame
df_dec1 = pd.read_csv('c:/Python311/Lib/site-packages/yahoo_fin/dataset/28Nov/SPXW US -01-12-2023_data-Copy.csv')


# Apply the function to each row of the DataFrame
df_dec1['IV'] = df_dec1.apply(lambda row: calculate_iv(BSParams(underlyingPrice= spot, strikePrice= row['strike'], interestRate= r, 
                                 daysToExpiration = forward_days, price = row['price'], option_type = row['option_type'])), axis=1)

df_dec1['calc_price_list'] = df_dec1.apply(lambda row: calculate_price(BSParams(underlyingPrice= spot, strikePrice= row['strike'], interestRate= r,
                                            daysToExpiration =  forward_days, volatility = row['IV'])), axis=1)
df_dec1['concatenated'] = df_dec1['calc_price_list'].apply(lambda x: ':'.join(map(str, x)))
display(HTML(df_dec1.to_html(index=False, border=0)))



11 15840


c:\Python311\Lib\site-packages\yahoo_fin\stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


NameError: name 'cubic_spline_risk_free_rate' is not defined

In [32]:
import pandas as pd
import numpy as np
from py_vollib_vectorized.black_scholes import implied_volatility

# Parameters
s = 100  # Underlying price
t = 0.5  # Time to expiration
r = 0.01  # Risk-free rate

# Assuming df is your DataFrame and it has columns 'option_prices', 'flag', 's', 'k'
df = pd.DataFrame({
    'option_prices': [11.70, 7.95, 5.00, 3.05, 1.90],
    'flag': ['c', 'c', 'c', 'c', 'c'],
    's': [100, 100, 100, 100, 100],
    'k': [90, 95, 100, 105, 110]
})


def calculate_iv(row):
    return implied_volatility(row['option_prices'], row['flag'], row['s'], row['k'], t, r)


df['implied_volatility'] = df.apply(calculate_iv, axis=1)

print(df)


ModuleNotFoundError: No module named 'py_vollib_vectorized'

In [19]:
%pip install py_vollib_vectorized



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached py_vollib_vectorized-0.1.1-py3-none-any.whl (30 kB)
  Using cached py_vollib-1.0.1-py3-none-any.whl


In [6]:
import datetime
import pytz


def chicago_eod_four_PM(eod, near_term=True):
    # Create a datetime object for 4 PM on November 10
    # replace 2023 with the correct year
    # 14.17 as of 11th Nov 2020
    four_pm = datetime.datetime(
        eod['year'], eod['month'], eod['day'], eod['hour'], eod['minute'], 0)  # 10th Nov 2023

    # Convert the datetime object to Eastern Time
    eastern = pytz.timezone('US/Eastern')
    four_pm_et = eastern.localize(four_pm)
    custom_print("four_pm_et: ", four_pm_et)

    tomorrow = datetime.datetime(
        four_pm_et.year, four_pm_et.month, four_pm_et.day, tzinfo=four_pm_et.tzinfo) + datetime.timedelta(1)
    days = [four_pm_et + datetime.timedelta(index) for index in range(24, 38)]

    friday = next(day for day in days if day.weekday() == 4)
    next_term = -1 if near_term == True else 6
    days_to_expiration = (friday - four_pm_et).days + next_term
    expiration_day = datetime.datetime(
        four_pm_et.year, four_pm_et.month, four_pm_et.day,
        tzinfo=four_pm_et.tzinfo) + datetime.timedelta(days_to_expiration + 1)
    custom_print("days to expiration: ", days_to_expiration)
    custom_print("friday: ", friday)
    custom_print("expiration day: ", expiration_day)
    custom_print("expiration day - calculation day: ", expiration_day - four_pm_et)

    # minutes_to_settlement = 570 if friday.day in range(     #friday.day to expiration_day.day will cover near and next term
    #     15, 22) and friday.weekday() == 4 else 960

    minutes_to_settlement = 570 if expiration_day.day in range(
        15, 22) and friday.weekday() == 4 else 960

    minutes_to_midnight_today = (tomorrow - four_pm_et).seconds // 60
    minutes_to_expiration = days_to_expiration * 24 * 60

    custom_print(minutes_to_midnight_today, minutes_to_settlement, minutes_to_expiration)
    time_to_expiration = (minutes_to_midnight_today +
                          minutes_to_settlement + minutes_to_expiration)

    # time_to_expiration = (minutes_to_midnight_today +
    #                       minutes_to_settlement + minutes_to_expiration) / 525600

    return (time_to_expiration, expiration_day, four_pm_et, days_to_expiration)


yesterday = datetime.datetime(2023, 1, 16) - datetime.timedelta(days=0)
eod = {
    "year": yesterday.year,
    "month": yesterday.month,
    "day": yesterday.day,
    "hour": 16,
    "minute": 15,
    "second": 0
}
print("Time to expiration: ", chicago_eod_four_PM(
    eod, True)[1], chicago_eod_four_PM(eod, False)[1])


Time to expiration:  2023-02-10 00:00:00-05:00 2023-02-17 00:00:00-05:00


In [7]:
import json
import requests
import re


def scrape_us_treasury_yield_curve_and_transpose_it_to_minutes_scale(days = 0):
    """
    Retrieves and formats the US Treasury yield curve.
    No arguments.
    """
    calctime = datetime.datetime.now() - datetime.timedelta(days)
    year = calctime.year
    month = calctime.month
    custom_print(year, month)
    # year = datetime.datetime.now().year
    # month = datetime.datetime.now().month
    url = f"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/pages/xmlview?data=daily_treasury_yield_curve&field_tdr_date_value_month={year}{month:02d}"
    response = requests.get(url)

    #print(json.dumps(response.content.decode(), indent=4))
    root = ET.fromstring(response.content)

    temp_us_treasury_dict = {}
    us_treasury_dict = {}

    for elt in root.iter():
        temp_us_treasury_dict[elt.tag[-8:]] = elt.text

    for key in temp_us_treasury_dict.keys():
        if (key.find("MONTH") + key.find("YEAR") + key.find("DATE") != -3):
            us_treasury_dict[re.sub(r'.*_', '', key)
                             ] = temp_us_treasury_dict[key]

    # Based on the average number of day in a month: 30.42
    minutes_in_a_month = 43804
    minutes_in_a_year = 525600

    # minute axis
    #x = [int(key[:-5])*minutes_in_a_month if "MONTH" in key else int(key[:-4])*minutes_in_a_year if "YEAR" in key else None for key in us_treasury_dict.keys()]
    x = [int(key[:-5])*minutes_in_a_month if "MONTH" in key else int(key[:-4]) *
         minutes_in_a_year if "YEAR" in key else None for key in us_treasury_dict.keys() if "MONTH" in key or "YEAR" in key]

    # yield axis
    y = [us_treasury_dict[key]
         for key in us_treasury_dict.keys() if "MONTH" in key or "YEAR" in key]
    custom_print("Yield curve:", us_treasury_dict)
    custom_print("Yield curve in minutes scale:", dict(zip(x, y)))

    return [x, y]

r = scrape_us_treasury_yield_curve_and_transpose_it_to_minutes_scale(days = 365)
r

[[43804,
  87608,
  131412,
  175216,
  262824,
  525600,
  1051200,
  1576800,
  2628000,
  3679200,
  5256000,
  10512000,
  15768000],
 ['4.58',
  '4.64',
  '4.70',
  '4.74',
  '4.80',
  '4.68',
  '4.21',
  '3.90',
  '3.63',
  '3.59',
  '3.52',
  '3.78',
  '3.65']]

In [8]:
import numpy as np
from scipy.interpolate import CubicSpline
from datetime import date
import math


def cubic_spline_risk_free_rate(minutes_to_expiration, days = 0):
    """
    Estimates the risk free rate (based on the US treasury yield) at a specific time to expiration
    :param <time_to_expiration>: integer ; time to expiration in day or minutes 
    :param <minute_data>: boolean ; indicates if the argument <time_to_expiration> is in minutes or days
    """

    # Extract the tenors and corresponding yields from the US Treasury yield curve
    yc = scrape_us_treasury_yield_curve_and_transpose_it_to_minutes_scale()
    tenors = yc[0]
    rates = yc[1]

    # Sort the data by tenor
    sorted_indices = np.argsort(tenors)
    # print(sorted_indices)
    #tenors_sorted = tenors[sorted_indices.tolist()]
    #rates_sorted = rates[sorted_indices.tolist()]

    tenors_sorted = [tenors[i] for i in sorted_indices]
    rates_sorted = [rates[i] for i in sorted_indices]

    # print(tenors_sorted)
    # print(rates_sorted)

    # Create a cubic spline interpolator
    cs = CubicSpline(tenors_sorted, rates_sorted,
                     extrapolate=True)  # Allow extrapolation

    # Specify expiration dates of near-term and next-term options (replace with actual dates)
    #near_term_expiration = date(2023, 12, 31)
    #next_term_expiration = date(2024, 3, 31)

    # Calculate time remaining until expiration in years
    #today = date.today()
    #time_to_near_term = (near_term_expiration - today).days / 365
    #time_to_next_term = (next_term_expiration - today).days / 365

    time_to_near_term = minutes_to_expiration[0]
    time_to_next_term = minutes_to_expiration[1]
    #time_to_near_term = 34,484
    #time_to_next_term = 44,954

    # Calculate interpolated/extrapolated yields using cubic spline
    yield_R1 = cs(time_to_near_term)
    yield_R2 = cs(time_to_next_term)
    # print(f"R1 (BEY) for {time_to_near_term} minutes: {yield_R1}")
    # print(f"R2 (BEY) for {time_to_next_term} minutes: {yield_R2}")

    # Convert BEY to APY using the correct formula
    APY_R1 = ((1 + yield_R1 / 2) ** 2) - 1
    APY_R2 = ((1 + yield_R2 / 2) ** 2) - 1

    r_near = np.log(1 + APY_R1)
    r_next = np.log(1 + APY_R2)

    # print(f"R1 (APY) for {time_to_near_term} minutes: {APY_R1}")
    # print(f"R2 (APY) for {time_to_next_term} minutes: {APY_R2}")

    # print(f"R1 (ln(APY+1)) for {time_to_near_term} minutes: {r_near}")
    # print(f"R2 (ln(APY+1)) for {time_to_next_term} minutes: {r_next}")

    # return [APY_R1, APY_R2]
    # return [r_near, r_next]
    return [yield_R1, yield_R2]


In [9]:
def add_strike_diff(df):
    # Create a new column 'strike_prev' that contains the previous 'strike' values
    df['strike_prev'] = df['strike'].shift(-1)

    # Create a new column 'strike_next' that contains the next 'strike' values
    df['strike_next'] = df['strike'].shift(1)

    # Create a new column 'strike_diff' that contains the differences between 'strike_next' and 'strike_prev'
    df['strike_diff'] = (df['strike_prev'] - df['strike_next'])/2

    # For the first and last 'strike', keep the original 'strike' values
    # df.loc[0, 'strike_diff'] = df.loc[1, 'strike'] - df.loc[0, 'strike']
    df.iloc[0, df.columns.get_loc('strike_diff')] = df.iloc[1, df.columns.get_loc('strike')] - \
        df.iloc[0, df.columns.get_loc('strike')]

    df.iloc[-1, df.columns.get_loc('strike_diff')] = df.iloc[-1, df.columns.get_loc('strike')] -\
        df.iloc[-2, df.columns.get_loc('strike')]
    # df.loc[df.index[-1], 'strike_diff'] = df.loc[df.index[-1],
    #                                              'strike'] - df.loc[df.index[-2], 'strike']
    return df


In [10]:
def remove_strikes(df, option_type = None):
    # Create new columns that contain the 'bid' values of the preceding and following rows
    df['bid_prev1'] = df['bid'].shift(1)
    df['bid_next1'] = df['bid'].shift(-1)

    # For 'Call' options, find the first index where 'bid' and 'bid_next1' are 0
    if option_type == 'Call' or df['option_type'].isin(['C', 'Call']).any():
        try:
            index_to_drop = df[(df['bid'] == 0) & (
                df['bid_next1'] == 0)].index[0]
            # Drop all rows from 'index_to_drop' to the end of the DataFrame
            df = df.loc[:index_to_drop-1]
        except IndexError:
            pass
        except KeyError as e:
        # Log the state of the DataFrame when the error occurs
            logger.info(f"Error: {e}")
            logger.info(f"DataFrame state: {df.to_string()}")
            logger.info(f"index_to_drop: {index_to_drop}")
            logger.info(f"last_index: {last_index}")
            #print(f"Error: {e}")
            #print(f"DataFrame state: {df.to_string()}")
            print(f"index_to_drop: {index_to_drop}")
            print(f"last_index: {last_index}")

    # For 'Put' options, find the first index where 'bid' and 'bid_prev1' are 0
    elif option_type == 'Put' or df['option_type'].isin(['P', 'Put']).any():
        try:
            index_to_drop = df[(df['bid'] == 0) & (
                df['bid_prev1'] == 0)].index[0]
            # Drop all rows from the start of the DataFrame to 'index_to_drop'
            last_index = df.index[-1]
            if index_to_drop < last_index:
                df = df.loc[index_to_drop+1:]
            else:
                df = df.loc[:last_index]
            #df = df.loc[index_to_drop+1:]
        except IndexError:
            pass
        except KeyError as e:
        # Log the state of the DataFrame when the error occurs
            logger.info(f"Error: {e}")
            logger.info(f"DataFrame state: {df.to_string()}")
            logger.info(f"index_to_drop: {index_to_drop}")
            logger.info(f"last_index: {last_index}")
            #print(f"Error: {e}")
            #print(f"DataFrame state: {df.to_string()}")
            print(f"index_to_drop: {index_to_drop}")
            print(f"last_index: {last_index}")

    # Drop the 'bid_prev1' and 'bid_next1' columns
    df = df.drop(columns=['bid_prev1', 'bid_next1'])

    # remove any row that has zeor bid
    df = df[df['bid'] != 0]

    #display(HTML(df.to_html(index=False, border=0)))

    return df


In [11]:
def forward_and_atm(min_diff_strike, df_calls, df_puts, activation, near_term):
    """
    Calculate forward VIX by extraploating minimum strike price...
    """
    df_calls_copy = df_calls.copy().set_index('strike', inplace=False)
    df_puts_copy = df_puts.copy().set_index('strike', inplace=False)

    forward = min_diff_strike + \
        activation * (df_calls.loc[df_calls['strike'] == min_diff_strike, 'mid-quote'].values[0] -
                      df_puts.loc[df_puts['strike'] == min_diff_strike, 'mid-quote'].values[0])

    print("Forward: ", forward)
    common_strikes = df_calls_copy.index.intersection(df_puts_copy.index)

    lower_strikes = common_strikes[common_strikes < forward]
    atm_strike = lower_strikes.max()

    #atm_strike = df_calls[df_calls['strike'] < f_near]['strike'].max()

    row_calls = df_calls.loc[df_calls['strike']
                             == atm_strike].reset_index(drop=True)
    row_puts = df_puts.loc[df_puts['strike']
                           == atm_strike].reset_index(drop=True)
    average_values = (row_calls[['mid-quote', 'bid', 'ask', 'px_last']] +
                      row_puts[['mid-quote', 'bid', 'ask', 'px_last']]) / 2
    average_values['strike'] = atm_strike
    average_values['option_type'] = 'ATM Avg Put/Call'
    atm_df = average_values[['strike', 'bid', 'ask',
                             'option_type', 'px_last', 'mid-quote']]

    display(HTML(atm_df.to_html(index=False, border=0)))
    return forward, atm_df, atm_strike


In [12]:
def find_min_diff_strike(df_calls, df_puts):
    # Set the strike price as the index in both dataframes
    df_calls_copy = df_calls.copy()
    df_puts_copy = df_puts.copy()

    df_calls_copy.set_index('strike', inplace=True)
    df_puts_copy.set_index('strike', inplace=True)

    # Calculate the difference between the mid-quote values

    mid_quote_diff = (df_calls_copy['mid-quote'] -
                      df_puts_copy['mid-quote']).abs()

    # Find the strike price with the minimum difference
    try:
        min_diff_strike = mid_quote_diff.idxmin()
    except ValueError:
        logger.error("Attempted to get argmin of an empty sequence.")
        logger.error(f"mid_quote_diff: {mid_quote_diff}")
        logger.error(f"df_calls_copy: {df_calls_copy} df_puts_copy: {df_puts_copy}")
        min_diff_strike = None

    custom_print("min_diff_strike: ", min_diff_strike)
    return min_diff_strike


In [13]:
def find_min_diff_strike_new(df_calls, df_puts, activation, near_term):
    # Set the strike price as the index in both dataframes
    df_calls_local = df_calls.copy()
    df_puts_local = df_puts.copy()

    df_calls_local['strike-copy'] = df_calls_local['strike']
    df_puts_local['strike-copy'] = df_puts_local['strike']

    df_calls_local.set_index('strike', inplace=True)
    df_puts_local.set_index('strike', inplace=True)

    # Find common strikes
    common_strikes = df_calls_local.index.intersection(df_puts_local.index)

    # Filter dataframes to only include common strikes
    df_calls_local = df_calls_local.loc[common_strikes]
    df_puts_local = df_puts_local.loc[common_strikes]

    # Filter out rows where mid-quote is zero in both dataframes
    non_zero_mask = ~((df_calls_local['mid-quote'] == 0)
                      & (df_puts_local['mid-quote'] == 0))
    df_calls_local = df_calls_local[non_zero_mask]
    df_puts_local = df_puts_local[non_zero_mask]

    # Calculate the difference between the mid-quote values
    mid_quote_diff = (
        df_calls_local['mid-quote'] - df_puts_local['mid-quote']).abs()

    # Find the strike price with the minimum difference
    try:
        min_diff_strike = mid_quote_diff.idxmin()
    except ValueError:
        logger.error("Attempted to get argmin of an empty sequence.")
        logger.error(f"mid_quote_diff: {mid_quote_diff}")
        logger.error(f"df_calls_copy: {df_calls_copy} df_puts_copy: {df_puts_copy}")
        min_diff_strike = None    
    custom_print("min_diff_strike: ", min_diff_strike)

    #######################################################################################
    forward = min_diff_strike + \
        activation * (df_calls.loc[df_calls['strike'] == min_diff_strike, 'mid-quote'].values[0] -
                      df_puts.loc[df_puts['strike'] == min_diff_strike, 'mid-quote'].values[0])

    custom_print("Forward: ", forward)

    atm_strike_another = df_calls_local.loc[df_calls_local['strike-copy']
                                            < forward, 'strike-copy'].max()
    custom_print("atm_strike-another: ", atm_strike_another)

    atm_strike = df_calls_local[df_calls_local['strike-copy']
                                < forward]['strike-copy'].max()

    custom_print("atm_strike: ", atm_strike)
    ###display(HTML(df_calls_local.to_html(index=False, border=0)))
    ###display(HTML(df_puts_local.to_html(index=False, border=0)))

    row_calls = df_calls.loc[df_calls['strike']
                             == atm_strike].reset_index(drop=True)
    row_puts = df_puts.loc[df_puts['strike']
                           == atm_strike].reset_index(drop=True)
    average_values = (row_calls[['mid-quote', 'bid', 'ask', 'px_last']] +
                      row_puts[['mid-quote', 'bid', 'ask', 'px_last']]) / 2
    average_values['strike'] = atm_strike
    average_values['option_type'] = 'ATM Avg Put/Call'
    atm_df = average_values[['strike', 'bid', 'ask',
                             'option_type', 'px_last', 'mid-quote']]

    #######display(HTML(atm_df.to_html(index=False, border=0)))
    return forward, atm_df, atm_strike
    ##############################################################################################

    # return min_diff_strike


In [14]:
def preapare_data(df):
    df = df.fillna(0)
    # Split the dataframe into calls and puts
    df_calls = df[df['option_type'] == 'C'].copy()
    df_puts = df[df['option_type'] == 'P'].copy()

    df_puts.loc[:, 'mid-quote'] = (df_puts['bid'] + df_puts['ask']) / 2
    df_calls.loc[:, 'mid-quote'] = (df_calls['bid'] + df_calls['ask']) / 2

    return df_calls, df_puts


In [17]:
import pandas as pd
from IPython.display import display, HTML


def chain_of_responsibility(df, r, t, near_term=True):

    df = df.fillna(0)

    df_calls, df_puts = preapare_data(df)

    # Remove the strikes with no bid price
    df_puts, df_calls = [remove_strikes(
        df_puts, 'Put'), remove_strikes(df_calls, 'Call')]

    # if near_term is False:
    #     display(HTML(df_puts.to_html(index=False, border=0)))
    #     display(HTML(df_calls.to_html(index=False, border=0)))

    # Find the min strike difference
    ######min_diff_strike = find_min_diff_strike_new(df_calls, df_puts, near_term)

    ###########################
    activation = np.exp(r*t)
    forward, df_atm, atm_strike = find_min_diff_strike_new(
        df_calls, df_puts, activation, near_term)
    ##########################

    # Calculate Forwrard
    # activation = np.exp(r*t)
    # forward, df_atm, atm_strike = forward_and_atm(
    #     min_diff_strike, df_calls.copy(), df_puts.copy(), activation, near_term)

    # activation = np.exp(r*t)
    # forward, df_atm, atm_strike = calculate_forward(
    #     min_diff_strike, df_calls, df_puts, activation, near_term)

    # Filter the dataframe to include only OTM strike
    df_puts = df_puts[df_puts['strike'] < atm_strike]
    df_calls = df_calls[df_calls['strike'] > atm_strike]

    # Remove the strikes with no bid price
    df_puts, df_calls = [remove_strikes(
        df_puts, 'Put'), remove_strikes(df_calls, 'Call')]

    # Combine data frames
    df_otm = pd.concat([df_puts, df_atm, df_calls], ignore_index=True)
    df_otm = df_otm.sort_values('strike')

    # Add adjescent strike difference
    df_otm = add_strike_diff(df_otm)

    # Contribution per Strike
    df_otm['strike_squared'] = df_otm['strike'] ** 2
    df_otm['strike_contribution'] = (
        df_otm['strike_diff'] * df_otm['mid-quote'] * activation)/df_otm['strike_squared']

    #display(HTML(df_otm.to_html(index=False, border=0)))
    total_contribution = 2 * df_otm['strike_contribution'].sum()/t
    #print("Total contribution", total_contribution)
    decay = ((forward/atm_strike - 1) ** 2)/t

    sigma_squared = (total_contribution - decay)
    ################################Per strike Quantity Contribution####################################
    df_otm['per_strike_qunatity_contribution'] = (df_otm['strike_diff'] * activation)/df_otm['strike_squared']
    df_otm['per_strike_quantity_term_adjustment'] = (2 * df_otm['per_strike_qunatity_contribution'])/t
    ###################################################################################################
    
    ######print("Sigma squared", sigma_squared)
    #display(HTML(df_otm.to_html(index=False, border=0)))
    return sigma_squared, df_otm, atm_strike, forward


In [28]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML

def create_dataset_from_data_stream_fut(expiry_date, instrument_type = 'option', req_type = 'quote', sec = 'SPXW'):
    # Define the base URL
    base_url = "http://127.0.0.1:25510/bulk_snapshot"
    request_url = f"{base_url}/{instrument_type}/{req_type}"
    params = {
        
        "exp": expiry_date,    
        "root": sec
    }
    # Send the request
    response = requests.get(request_url, params=params, headers={'Accept': 'application/json'})

    data = json.loads(response.text)
    df = pd.json_normalize(data['response'])
    tick_columns = ["ms_of_day", "bid_size", "bid_exchange", "bid", "bid_condition", "ask_size", "ask_exchange", "ask", "ask_condition", "date"]
    try:
        # Create a new DataFrame from the 'tick' column
        tick_df = pd.DataFrame(df['tick'].tolist(), columns=tick_columns)

        # Drop the 'tick' column from the original DataFrame
        df = df.drop('tick', axis=1)

        # Concatenate the original DataFrame with the new DataFrame
        df = pd.concat([df, tick_df], axis=1)

        df['strike'] = df['contract.strike']/1000
        df.sort_values(by=['strike'], inplace=True)
        df = df.rename(columns={'contract.right': 'option_type'})
        working_set = df[['strike', 'bid', 'ask', 'option_type']].copy()
        working_set.loc[:, 'px_last'] = 999
        ###working_set['px_last'] = 999
    except Exception as e:
        # Log the error message and the program state
        logger.error(f"Error: {str(e)}")
        logger.error(f"Expiry date: {expiry_date}")
        logger.error(f"Start date: {start_date}")
        logger.error(f"Program state: {df}")
        return None

    #working_set.to_csv('./historical/output.csv', index=False)
    #working_set.to_json('./historical/output.json', index=False)
    ######display(HTML(working_set.to_html(index=False, border=0)))

    # Return the response
    return working_set

working_set = create_dataset_from_data_stream_fut(expiry_date, instrument_type = 'option', req_type = 'quote', sec = 'SPXW')

NameError: name 'start_date' is not defined

In [63]:
import requests
import json
import pandas as pd
from datetime import datetime
import numpy as np
from IPython.display import display, HTML

def create_dataset_from_data_stream(instrument_type, req_type, sec, start_date, end_date, expiry_date, ivl):
    # Define the base URL
    base_url = "http://127.0.0.1:25510/bulk_at_time"

    # Construct the request URL
    request_url = f"{base_url}/{instrument_type}/{req_type}"

    # Define the query parameters
    params = {
        "end_date": end_date,
        "exp": expiry_date,
        "ivl": ivl,
        "root": sec,
        "start_date": start_date
    }

    # Send the request
    response = requests.get(request_url, params=params)
    data = json.loads(response.text)
    df = pd.json_normalize(data['response'])
    tick_columns = ["ms_of_day", "bid_size", "bid_exchange", "bid", "bid_condition", "ask_size", "ask_exchange", "ask", "ask_condition", "date"]
    try:
        df[tick_columns] = df['ticks'].apply(lambda x: pd.Series(x[0]))
        df['strike'] = df['contract.strike']/1000
        df.sort_values(by=['strike'], inplace=True)
        df = df.rename(columns={'contract.right': 'option_type'})
        working_set = df[['strike', 'bid', 'ask', 'option_type']].copy()
        working_set.loc[:, 'px_last'] = 999
        ###working_set['px_last'] = 999
        

    except Exception as e:
        # Log the error message and the program state
        logger.error(f"Error: {str(e)}")
        logger.error(f"Expiry date: {expiry_date}")
        logger.error(f"Start date: {start_date}")
        logger.error(f"Program state: {df}")
        return None
        

    #working_set.to_csv('./historical/output.csv', index=False)
    #working_set.to_json('./historical/output.json', index=False)
    #display(HTML(working_set.to_html(index=False, border=0)))

    # Return the response
    return working_set

trade_date, end_date, expiry_date = '20230214', '20230214', '20230215' 
working_set = create_dataset_from_data_stream('option', 'quote', 'SPXW', trade_date, end_date, expiry_date, 57675000)

In [36]:
import datetime
# Time to expiration
import vix_utils

def spawn_backtest(vix_future_settlement_date, calc_date):
    eod = {
        "year": vix_future_settlement_date.year,
        "month": vix_future_settlement_date.month,
        "day": vix_future_settlement_date.day,
        "hour": 16,
        "minute": 15,
        "second": 0
    }

    delta = abs(datetime.datetime.now() - calc_date)
    ####print(f"Time delta : {delta} ")
    custom_print("delta: ", delta)


    l_t1, l_t2 = chicago_eod_four_PM(
        eod, near_term=True), chicago_eod_four_PM(eod, near_term=False)

    start_date = calc_date.strftime('%Y%m%d')
    end_date =   calc_date.strftime('%Y%m%d')

    option_expiry_near = l_t1[1].strftime('%Y%m%d')
    option_expiry_next = l_t2[1].strftime('%Y%m%d')
    
    forward_days = (vix_future_settlement_date - calc_date).days * 1440

    n_t1 = l_t1[0] + forward_days
    n_t2 = l_t2[0] + forward_days

    print(f"n_t1 and n_t2 {n_t1} {n_t2} ")

    # extrapolated rates to expiration
    r_near, r_next = np.array(cubic_spline_risk_free_rate(
        [n_t1, n_t2], days = delta))/100
    custom_print("r_near and r_next", r_near, r_next, )
    custom_print(f"start_date : {start_date}, end_date : {end_date}, option_expiry_near : {option_expiry_near}, option_expiry_next : {option_expiry_next}")
    print(f"start_date : {start_date}, end_date : {end_date}, option_expiry_near : {option_expiry_near}, option_expiry_next : {option_expiry_next}")
    t_near = n_t1/525600
    t_next = n_t2/525600

    df_near = create_dataset_from_data_stream("option", "quote", "SPXW", start_date = start_date, 
                                             end_date = start_date, expiry_date = option_expiry_near, ivl = 57675000)


    df_next = create_dataset_from_data_stream("option", "quote", "SPXW", start_date = start_date, 
                                             end_date = start_date, expiry_date = option_expiry_next, ivl = 57675000)
    
    
    if df_near is None or df_next is None:
        return None
                                        
    sigma_squared_near, df_vix_near, atm_near, forward_near = chain_of_responsibility(df_near, r_near, t_near) 
    sigma_squared_next, df_vix_next, atm_next,forward_next = chain_of_responsibility(df_next, r_next, t_next, False)
    
    atm_near_details = df_vix_near.loc[df_vix_near['strike'] == atm_near]
    atm_next_details = df_vix_next.loc[df_vix_next['strike'] == atm_next]                                        

    custom_print("Sigma squared near", sigma_squared_near)
    custom_print("Sigma squared next", sigma_squared_next)

    n_30 = 43200 + forward_days
    n_365 = 525600
    n_y_m = n_365/n_30

    tnear_sig_squared = sigma_squared_near * t_near
    tnext_sig_squared = sigma_squared_next * t_next


    near_dev = tnear_sig_squared * ((n_t2 - n_30)/(n_t2 - n_t1))
    next_dev = tnext_sig_squared * ((n_30 - n_t1)/(n_t2 - n_t1))
    tot_dev = near_dev + next_dev
    vix = np.sqrt(tot_dev * n_y_m) * 100

    print(f'Vix Forward on {start_date} - {vix}')
    #######print(f'atm_near - {atm_near}')
    ########print(f'atm_next - {atm_next}')

    ################################################
    df_vix_near['T1'] = t_near
    df_vix_near['(M_t2 - M_30)/(M_t2 - M_t1)'] = (n_t2 - n_30)/(n_t2 - n_t1)
    df_vix_near['M_365/M_30'] = n_y_m

    df_vix_near['per_strike_constants'] = df_vix_near['per_strike_quantity_term_adjustment'] * n_y_m * t_near * (n_t2 - n_30)/(n_t2 - n_t1)
 
    df_vix_next['T2'] = t_next
    df_vix_next['(M_30 - M_t1)/(M_t2 - M_t1)'] = (n_30 - n_t1)/(n_t2 - n_t1)
    df_vix_next['M_365/M_30'] = n_y_m

    df_vix_next['per_strike_constants'] = df_vix_next['per_strike_quantity_term_adjustment'] * n_y_m * t_next * (n_30 - n_t1)/(n_t2 - n_t1)

    ###############################################

    return vix, df_near, df_next, df_vix_near, df_vix_next, forward_near, forward_next, start_date, option_expiry_near, option_expiry_next

c:\Python311\Lib\site-packages\vix_utils\download_vix_futures.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  self.valid_days_set=frozenset(d.date() for d in self.valid_days.dt.to_pydatetime())


In [38]:
import os

def save_df_to_csv(df, path):
    # Extract directory from the path
    dir_path = os.path.dirname(path)

    # Check if the directory exists
    if not os.path.exists(dir_path):
        # If the directory doesn't exist, create it
        os.makedirs(dir_path)

    # Save the DataFrame to a CSV file
    df.to_csv(path, index=False)  

In [66]:
import os

def create_directory_structure(trade_date):
    # Define the directories to be created
    directories = [
        f'./result-set/{trade_date}',
        f'./result-set/{trade_date}/future',
        f'./result-set/{trade_date}/P&L',
        f'./result-set/{trade_date}/P&L/Delta-25&Above',
        f'./result-set/{trade_date}/spot'
    ]

    # Iterate over the directories
    for directory in directories:
        # Check if the directory does not already exist
        if not os.path.exists(directory):
            # Create the directory
            os.makedirs(directory)

In [68]:
###Backtesting for a single date
import datetime

def run_backtest(year, month, day):
    try:
        ins_type = "future"
        
        v = vix_utils.vix_futures_expiry_date_from_trade_date(year, month, day, 1) if ins_type == "future" else datetime.datetime(year, month, day)
        on_day = datetime.datetime.combine(datetime.datetime(year, month, day), datetime.time(16, 15, 0))
        vix_future_settlement_day = datetime.datetime.combine(v, datetime.time())
        print(f"Replicating Vix {ins_type} for {v} on {on_day}")
        
        create_directory_structure(on_day.strftime('%Y-%m-%d'))
        vix, df_near, df_next, df_vix_near, df_vix_next, forward_near, forward_next, start_date, option_expiry_near, option_expiry_next = spawn_backtest(vix_future_settlement_day, on_day)
        
        df_near['mid_quote'] = (df_near['bid'] + df_near['ask']) / 2
        df_next['mid_quote'] = (df_next['bid'] + df_next['ask']) / 2
        #######################################################
        df_per_strike_quantity_near = df_vix_near[['strike', 'option_type', 'mid-quote', 'per_strike_quantity_term_adjustment',\
                                                    'T1', '(M_t2 - M_30)/(M_t2 - M_t1)', 'M_365/M_30', 'per_strike_constants' ]].copy()
        df_per_strike_quantity_next = df_vix_next[['strike', 'option_type', 'mid-quote', 'per_strike_quantity_term_adjustment',\
                                                    'T2', '(M_30 - M_t1)/(M_t2 - M_t1)', 'M_365/M_30', 'per_strike_constants']].copy()
        
        df_per_strike_quantity_near = df_per_strike_quantity_near.rename(columns={'mid-quote': 'mid_quote'})
        df_per_strike_quantity_next = df_per_strike_quantity_next.rename(columns={'mid-quote': 'mid_quote'})
        df_per_strike_quantity_near['vix_forward'] = vix
        df_per_strike_quantity_next['vix_forward'] = vix
        observation_date = on_day.strftime('%Y-%m-%d')
        save_df_to_csv(df_per_strike_quantity_near, f'./result-set/{observation_date}/{ins_type}/dataset_{option_expiry_near}.csv')
        save_df_to_csv(df_per_strike_quantity_next, f'./result-set/{observation_date}/{ins_type}/dataset_{option_expiry_next}.csv')

        if vix_future_settlement_day.date() == on_day.date():
            save_df_to_csv(df_near, f'./result-set/{observation_date}/{ins_type}/dataset_complete_{option_expiry_near}.csv')
            save_df_to_csv(df_next, f'./result-set/{observation_date}/{ins_type}/dataset_complete_{option_expiry_next}.csv')
        ########################################################

        
    
    except Exception as e:
        print(f"Error occurred on {year}-{month}-{day}: {e}")
        return None

results = {}
business_days = pd.bdate_range(start='2023-02-01', end='2023-03-22')
for day in business_days:
    result = run_backtest(day.year, day.month, day.day)
    

Replicating Vix future for 2023-02-15 on 2023-02-01 16:15:00
n_t1 and n_t2 61515 71985 
start_date : 20230201, end_date : 20230201, option_expiry_near : 20230317, option_expiry_next : 20230324
Vix Forward on 20230201 - 19.565824841154186
Replicating Vix future for 2023-02-15 on 2023-02-02 16:15:00
n_t1 and n_t2 60075 70545 
start_date : 20230202, end_date : 20230202, option_expiry_near : 20230317, option_expiry_next : 20230324
Vix Forward on 20230202 - 19.88614199609086
Replicating Vix future for 2023-02-15 on 2023-02-03 16:15:00
n_t1 and n_t2 58635 69105 
start_date : 20230203, end_date : 20230203, option_expiry_near : 20230317, option_expiry_next : 20230324
Vix Forward on 20230203 - 19.813355318359154
Replicating Vix future for 2023-02-15 on 2023-02-06 16:15:00
n_t1 and n_t2 54315 64785 
start_date : 20230206, end_date : 20230206, option_expiry_near : 20230317, option_expiry_next : 20230324
Vix Forward on 20230206 - 20.598331845159784
Replicating Vix future for 2023-02-15 on 2023-02-

In [41]:
def find_closest_delta_r(df):
    def find_option(option_type, target_delta, attribute_name):
        options = df[df['option_type'] == option_type]
        closest_option_index = (options['Delta'] - target_delta).abs().idxmin()
        df.loc[closest_option_index, 'Strike_attribute'] = attribute_name
        corresponding_option = df[(df['strike'] == df.loc[closest_option_index, 'strike']) & (df['option_type'] == ('C' if option_type == 'P' else 'P'))]
        if not corresponding_option.empty:
            df.loc[closest_option_index, 'other_strike_mid_quote'] = corresponding_option['mid_quote'].iloc[0]
        return df.loc[closest_option_index]

    closest_call = find_option('C', 0.25, '25 Delta Call')
    closest_put = find_option('P', -0.25, '25 Delta Put')

    return closest_call, closest_put

In [42]:
def find_closest_delta(df):
    # For call options, find the row with Delta closest to 0.25
    call_options = df[df['option_type'] == 'C']
    closest_call_index = (call_options['Delta'] - 0.25).abs().idxmin()
    df.loc[closest_call_index, 'Strike_attribute'] = '25 Delta Call'
    
    # For put options, find the row with Delta closest to -0.25
    put_options = df[df['option_type'] == 'P']
    closest_put_index = (put_options['Delta'] - (-0.25)).abs().idxmin()
    df.loc[closest_put_index, 'Strike_attribute'] = '25 Delta Put'
    
    # Find the corresponding mid quote for the closest call and put options
    corresponding_put = df[(df['strike'] == df.loc[closest_call_index, 'strike']) & (df['option_type'] == 'P')]
    if not corresponding_put.empty:
        df.loc[closest_call_index, 'other_strike_mid_quote'] = corresponding_put['mid-quote'].iloc[0]
        
    corresponding_call = df[(df['strike'] == df.loc[closest_put_index, 'strike']) & (df['option_type'] == 'C')]
    if not corresponding_call.empty:
        df.loc[closest_put_index, 'other_strike_mid_quote'] = corresponding_call['mid-quote'].iloc[0]
    
    return df.loc[closest_call_index], df.loc[closest_put_index]